<a href="https://colab.research.google.com/github/william-toscani/Data_Visualization_Project/blob/main/Data_Viz_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Libraries

In [ ]:
!pip install pandasql

In [2]:
import requests
import pandas as pd
import numpy as np
from pandasql import sqldf
sql = lambda q: sqldf(q, globals())

import plotly.express as px
import plotly.graph_objects as go

# Datasets

## Population

In [6]:
pop_raw = pd.read_csv("https://ourworldindata.org/grapher/population-with-un-projections.csv?v=1&csvType=full&useColumnShortNames=true", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})
#pop_metadata = requests.get("https://ourworldindata.org/grapher/population-with-un-projections.metadata.json?v=1&csvType=full&useColumnShortNames=true").json()

#display(pop_raw.head())
#display(pop_metadata)
#display(pop_df.info())
#display(pop_df.describe())
#display(pop_df.columns)
#display(pop_df.shape)
#display(pop_df.dtypes)
#display(pop_df.isnull().sum())

pop_clean = sql("""
SELECT Code, population__sex_all__age_all__variant_medium as population FROM pop_raw
WHERE Code IS NOT NULL AND Code IS NOT "OWID_WRL"
AND year = 2024
ORDER BY population__sex_all__age_all__variant_medium DESC
LIMIT 25
""")

display(pop_clean)

,Code,population
0,IND,1.450936e+09
1,CHN,1.419321e+09
2,USA,3.454266e+08
3,IDN,2.834879e+08
4,PAK,2.512692e+08
5,NGA,2.326795e+08
6,BRA,2.119986e+08
7,BGD,1.735624e+08
8,RUS,1.448204e+08
9,ETH,1.320598e+08


## World (connect countries with continent)

In [53]:
world_raw = pd.read_csv("https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv")
world_raw.rename(columns={'alpha-3': 'Code'}, inplace=True)
display(world_raw.head())
#display(world_raw.columns)


world_clean = sql("""
SELECT world_raw.Code, world_raw.region FROM world_raw
INNER JOIN pop_clean ON world_raw.Code = pop_clean.Code
""")

display(world_clean)

,name,alpha-2,Code,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN


,Code,region
0,BGD,Asia
1,BRA,Americas
2,CHN,Asia
3,COD,Africa
4,EGY,Africa
5,ETH,Africa
6,FRA,Europe
7,DEU,Europe
8,IND,Asia
9,IDN,Asia


## GDP

In [46]:
gdp_raw = pd.read_csv("https://ourworldindata.org/grapher/gdp-worldbank.csv?v=1&csvType=full&useColumnShortNames=true", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})
#gdp_metadata = requests.get("https://ourworldindata.org/grapher/gdp-worldbank.metadata.json?v=1&csvType=full&useColumnShortNames=true").json()

#display(gdp_raw.head())
#display(gdp_metadata)
#display(gdp_df.info())

gdp_clean = sql("""
SELECT gdp_raw.Code, gdp_raw.ny_gdp_mktp_pp_kd as gdp FROM gdp_raw
INNER JOIN pop_clean ON gdp_raw.Code = pop_clean.Code
WHERE gdp_raw.Code IS NOT NULL AND gdp_raw.Code IS NOT "OWID_WRL"
AND gdp_raw.year = 2024
""")

display(gdp_clean)

,Code,gdp
0,BGD,1472986459828
1,BRA,4165328804627
2,CHN,33597882047418
3,COD,164367209943
4,EGY,1957627322504
5,ETH,380895311983
6,FRA,3731760755094
7,DEU,5246975952526
8,IND,14243939416927
9,IDN,4102194303076


## Energy consumption by source

In [42]:
energy_cons_raw = pd.read_csv("https://ourworldindata.org/grapher/energy-consumption-by-source-and-country.csv?v=1&csvType=full&useColumnShortNames=true", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})
#metadata = requests.get("https://ourworldindata.org/grapher/energy-consumption-by-source-and-country.metadata.json?v=1&csvType=full&useColumnShortNames=true").json()

columns_to_sum = [col for col in energy_cons_raw.columns if col not in ['Year', 'Code', 'Entity']]
energy_cons_raw['total_consumption'] = energy_cons_raw[columns_to_sum].sum(axis=1)
display(energy_cons_raw)


energy_cons_clean = sql("""
SELECT Code, total_consumption FROM energy_cons_raw
WHERE Code IS NOT NULL AND Code IS NOT "OWID_WRL"
AND year = 2000
""")

display(energy_cons_clean)


,Entity,Code,Year,other_renewables_consumption_equivalent_twh,biofuels_consumption_twh,solar_consumption_equivalent_twh,wind_consumption_equivalent_twh,hydro_consumption_equivalent_twh,nuclear_consumption_equivalent_twh,gas_consumption_twh,coal_consumption_twh,oil_consumption_twh,total_consumption
0,Africa,NaN,1965,0.0,0.0,0.0,0.0,38.626762,0.0,9.571930,323.49615,341.07257,712.767412
1,Africa,NaN,1966,0.0,0.0,0.0,0.0,43.083344,0.0,10.698092,323.12222,369.46457,746.368225
2,Africa,NaN,1967,0.0,0.0,0.0,0.0,44.973988,0.0,10.573844,330.29160,368.28156,754.120992
3,Africa,NaN,1968,0.0,0.0,0.0,0.0,52.606503,0.0,10.717145,343.51288,389.57162,796.408148
4,Africa,NaN,1969,0.0,0.0,0.0,0.0,61.391357,0.0,12.520175,346.64288,397.37260,817.927012
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8312,Zimbabwe,ZWE,2020,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.000000
8313,Zimbabwe,ZWE,2021,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.000000
8314,Zimbabwe,ZWE,2022,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.000000
8315,Zimbabwe,ZWE,2023,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.000000


,Code,total_consumption
0,DZA,299.935970
1,AGO,0.000000
2,ARG,698.112469
3,AUS,1302.216344
4,AUT,387.558049
...,...,...
101,VEN,760.742325
102,VNM,211.541245
103,YEM,0.000000
104,ZMB,0.000000


## Co2 emissions

In [ ]:
emission_df = pd.read_csv("https://ourworldindata.org/grapher/annual-co2-emissions-per-country.csv?v=1&csvType=full&useColumnShortNames=true", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})
emission_metadata = requests.get("https://ourworldindata.org/grapher/annual-co2-emissions-per-country.metadata.json?v=1&csvType=full&useColumnShortNames=true").json()

display(emission_df.head())

,Entity,Code,Year,emissions_total
0,Afghanistan,AFG,1949,14656.0
1,Afghanistan,AFG,1950,84272.0
2,Afghanistan,AFG,1951,91600.0
3,Afghanistan,AFG,1952,91600.0
4,Afghanistan,AFG,1953,106256.0


# Data Analysis

## Research question: top 20 biggest countries in the world: how do they emit and consume energy?

In [ ]:
top_20_countries_df = sql("""
SELECT * FROM pop_df
WHERE Code IS NOT NULL AND Code IS NOT "OWID_WRL"
AND population__sex_all__age_all__variant_medium IS NOT NULL
AND year = 2024
ORDER BY population__sex_all__age_all__variant_medium DESC
LIMIT 25

""")

display(top_20_countries_df)

,Entity,Code,Year,population__sex_all__age_all__variant_estimates,population__sex_all__age_all__variant_medium
0,India,IND,2024,None,1.450936e+09
1,China,CHN,2024,None,1.419321e+09
2,United States,USA,2024,None,3.454266e+08
3,Indonesia,IDN,2024,None,2.834879e+08
4,Pakistan,PAK,2024,None,2.512692e+08
5,Nigeria,NGA,2024,None,2.326795e+08
6,Brazil,BRA,2024,None,2.119986e+08
7,Bangladesh,BGD,2024,None,1.735624e+08
8,Russia,RUS,2024,None,1.448204e+08
9,Ethiopia,ETH,2024,None,1.320598e+08


In [ ]:
new_gdp_df = sql("""
SELECT gdp_df.*, top_20_countries_df.population__sex_all__age_all__variant_medium AS TopCountryPopulation
FROM gdp_df
JOIN top_20_countries_df ON gdp_df.Code = top_20_countries_df.Code
WHERE gdp_df.Code IS NOT NULL AND gdp_df.Code IS NOT "OWID_WRL"
AND gdp_df.year = 2024

""")

display(new_gdp_df)

,Entity,Code,Year,ny_gdp_mktp_pp_kd,TopCountryPopulation
0,Bangladesh,BGD,2024,1472986459828,1.735624e+08
1,Brazil,BRA,2024,4165328804627,2.119986e+08
2,China,CHN,2024,33597882047418,1.419321e+09
3,Democratic Republic of Congo,COD,2024,164367209943,1.092763e+08
4,Egypt,EGY,2024,1957627322504,1.165383e+08
5,Ethiopia,ETH,2024,380895311983,1.320598e+08
6,France,FRA,2024,3731760755094,6.654853e+07
7,Germany,DEU,2024,5246975952526,8.455223e+07
8,India,IND,2024,14243939416927,1.450936e+09
9,Indonesia,IDN,2024,4102194303076,2.834879e+08


In [ ]:
emission_gdp_df = sql("""
SELECT
    new_gdp_df.Entity,
    new_gdp_df.Code,
    new_gdp_df.Year,
    new_gdp_df.TopCountryPopulation AS population,
    CAST(emission_df.emissions_total AS REAL) / new_gdp_df.ny_gdp_mktp_pp_kd AS emission_per_gdp
FROM new_gdp_df
JOIN emission_df ON new_gdp_df.Code = emission_df.Code AND new_gdp_df.Year = emission_df.Year
WHERE emission_df.Code IS NOT NULL AND emission_df.Code IS NOT "OWID_WRL"
AND emission_df.year = 2024

ORDER by population DESC
""")

display(emission_gdp_df)

,Entity,Code,Year,population,emission_per_gdp
0,India,IND,2024,1.450936e+09,0.000224
1,China,CHN,2024,1.419321e+09,0.000366
2,United States,USA,2024,3.454266e+08,0.000191
3,Indonesia,IDN,2024,2.834879e+08,0.000198
4,Pakistan,PAK,2024,2.512692e+08,0.000129
5,Nigeria,NGA,2024,2.326795e+08,0.000103
6,Brazil,BRA,2024,2.119986e+08,0.000116
7,Bangladesh,BGD,2024,1.735624e+08,0.000074
8,Russia,RUS,2024,1.448204e+08,0.000292
9,Ethiopia,ETH,2024,1.320598e+08,0.000047


In [ ]:
# Calculate min and max for 'emission_per_gdp'
min_emission = emission_gdp_df['emission_per_gdp'].min()
max_emission = emission_gdp_df['emission_per_gdp'].max()

# Apply min-max normalization
emission_gdp_df['emission_per_gdp_normalized'] = (emission_gdp_df['emission_per_gdp'] - min_emission) / (max_emission - min_emission)

display(emission_gdp_df)

,Entity,Code,Year,population,emission_per_gdp,emission_per_gdp_normalized
0,India,IND,2024,1.450936e+09,0.000224,0.378354
1,China,CHN,2024,1.419321e+09,0.000366,0.662845
2,United States,USA,2024,3.454266e+08,0.000191,0.311645
3,Indonesia,IDN,2024,2.834879e+08,0.000198,0.325698
4,Pakistan,PAK,2024,2.512692e+08,0.000129,0.187797
5,Nigeria,NGA,2024,2.326795e+08,0.000103,0.134867
6,Brazil,BRA,2024,2.119986e+08,0.000116,0.160841
7,Bangladesh,BGD,2024,1.735624e+08,0.000074,0.075587
8,Russia,RUS,2024,1.448204e+08,0.000292,0.515441
9,Ethiopia,ETH,2024,1.320598e+08,0.000047,0.021919


Research question: what are the major countries (in terms of gdp and population) for the electricity consumption via fossil?

In [ ]:
fossil = np.array(["oil_generation__twh_chart_electricity_prod_source_stacked","gas_generation__twh_chart_electricity_prod_source_stacked", "coal_generation__twh_chart_electricity_prod_source_stacked"])
renewables = np.array(["other_renewables_excluding_bioenergy_generation__twh_chart_electricity_prod_source_stacked", "bioenergy_generation__twh_chart_electricity_prod_source_stacked",
                       "hydro_generation__twh_chart_electricity_prod_source_stacked", "solar_generation__twh_chart_electricity_prod_source_stacked", "wind_generation__twh_chart_electricity_prod_source_stacked"])

fossil_elec_prod = sql(f"""
SELECT Entity, year, {",".join(fossil)} FROM elec_prod_df
WHERE CODE IS NOT NULL AND year > 1985
ORDER BY year
""")

display(fossil_elec_prod)


,Entity,Year,oil_generation__twh_chart_electricity_prod_source_stacked,gas_generation__twh_chart_electricity_prod_source_stacked,coal_generation__twh_chart_electricity_prod_source_stacked
0,Argentina,1986,7.894000,13.249956,1.013000
1,Australia,1986,3.771000,13.784000,92.587000
2,Brazil,1986,10.187736,0.000000,6.184231
3,Canada,1986,6.166000,6.886000,71.627000
4,China,1986,53.678000,1.693000,299.610350
...,...,...,...,...,...
5757,United Kingdom,2024,9.230000,84.160000,2.300000
5758,United States,2024,30.320000,1864.870000,652.760000
5759,Uruguay,2024,0.930000,0.000000,0.000000
5760,Vietnam,2024,0.110000,21.630000,149.350000
